# data에 있는 모든 문서 추출하기
- chroma_store가 없을 경우 data 폴더에 있는 pdf 파일들을 읽어서 벡터 DB에 저장 하기 위한 과정

In [17]:
from langsmith import utils
utils.get_env_var.cache_clear()

In [1]:
# pip install dotenv
import os
from dotenv import load_dotenv

load_dotenv()
LANGSMITH_API_KEY = os.getenv("LANGSMITH_API_KEY")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com" # 이건 그대로
os.environ["LANGCHAIN_API_KEY"] = LANGSMITH_API_KEY # 이거 API키 받은거
os.environ["LANGCHAIN_PROJECT"] = "lm-studio-rag-tracing"  # 프로젝트 이름 설정(아무거나 해도됨)
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

### 기본 방법 선택하기

In [67]:
# 청킹 방법
splitter_name = "semantic" # recursive, semantic

# LLM 모델
LLM_MODEL = "gpt-4o-mini" # gpt-4o-mini, openai/gpt-oss-120b

# 벡터 DB
VECTOR_DB = "faiss_index" # chroma_store, faiss_index

# 임베딩 모델
# EB_MODEL = "text-embedding-3-large" # text-embedding-3-large -> 아래 임베딩 모델 섹션에서 설정



## 임베딩 모델 설정하기
- 섹션이 바뀌기 전까지 셀 중 한가지 선택해서 사용.

### OpenAI Embeddings

In [68]:
# OpenAI Embeddings : text-embedding-3-large
# pip install langchain_openai
from langchain_openai import OpenAIEmbeddings

# 의미 기반으로 청킹을 하기 위해 OpenAI의 임베딩 모델을 사용
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
EB_MODEL = "text-embedding-3-large" # text-embedding-3-large
embedding = OpenAIEmbeddings(model=EB_MODEL, api_key=OPENAI_API_KEY)

### HuggingFaceEmbddings (local 저장)

In [80]:
# 로컬에 저장해서 사용하는거라 api key가 필요없음
# HuggingFaceEmbeddings : sentence-transformers/stsb-xlm-r-multilingual

# pip install langchain sentence-transformers langchain-community
from langchain_community.embeddings import HuggingFaceEmbeddings

# HuggingFace 임베딩 클래스 이용
EB_MODEL = "stsb-xlm-r-multilingual" # stsb-xlm-r-multilingual
embedding = HuggingFaceEmbeddings(model_name=EB_MODEL)

# 단일 텍스트 임베딩 생성
sample_text = "금융권 문서 임베딩 예제"
embedding_vector = embedding.embed_query(sample_text)

print("임베딩 벡터 길이:", len(embedding_vector))
print("임베딩 벡터 일부:", embedding_vector[:5])


임베딩 벡터 길이: 768
임베딩 벡터 일부: [0.11608216911554337, -0.2838129997253418, 0.600308895111084, -0.2887226939201355, 0.6230913400650024]


### Hugginface embeddings (multilinual-e5)

In [28]:
# pip install langchain_huggingface
from langchain_huggingface.embeddings import HuggingFaceEndpointEmbeddings

EB_MODEL = "intfloat/multilingual-e5-large-instruct"  # multilingual-e5-large, multilingual-e5-large-instruct
model_name = EB_MODEL

HF_API_KEY = os.getenv("HF_API_KEY")

embedding = HuggingFaceEndpointEmbeddings(
    model=model_name,
    task="feature-extraction",
    huggingfacehub_api_token=HF_API_KEY,
)

# 단일 텍스트 임베딩 생성
sample_text = "금융권 문서 임베딩 예제"
embedding_vector = embedding.embed_query(sample_text)

print("임베딩 벡터 길이:", len(embedding_vector))
print("임베딩 벡터 일부:", embedding_vector[:5])

임베딩 벡터 길이: 1024
임베딩 벡터 일부: [0.02028062380850315, 0.00727831618860364, -0.001797395758330822, -0.037471067160367966, 0.013377872295677662]


### Google Gemini Embeddings
- *호환성 문제로 genai 는 사용하지 않도록 한다.*

In [ ]:
# # pip install genai google google.genai
# from langchain.embeddings.base import Embeddings
# from google import genai
# from google.genai.types import EmbedContentConfig

# api_key = os.getenv("GC_API_KEY")

# client = genai.Client(api_key=api_key)
# EB_MODEL = "gemini-embedding-001"  # Google Cloud 임베딩 모델
# def call_google_cloud_embedding_api(text):
#     response = client.models.embed_content(
#         model=EB_MODEL,
#         contents=text,
#         config=EmbedContentConfig(
#             task_type="RETRIEVAL_DOCUMENT",
#             output_dimensionality=768
#         )
#     )
#     return response.embeddings[0].values

# class GoogleCloudEmbeddings(Embeddings):
#     def embed_documents(self, texts):
#         # Google Cloud 임베딩 API 호출 코드
#         # texts 리스트를 임베딩 벡터 리스트로 반환
#         embeddings = []
#         for text in texts:
#             embedding_vector = call_google_cloud_embedding_api(text)
#             embeddings.append(embedding_vector)
#         return embeddings

#     def embed_query(self, text):
#         return call_google_cloud_embedding_api(text)

# # 생성
# embedding = GoogleCloudEmbeddings()

# sample_text = "금융권 문서 임베딩 예제"
# embedding_vector = embedding.embed_query(sample_text)

# print("임베딩 벡터 길이:", len(embedding_vector))
# print("임베딩 벡터 일부:", embedding_vector[:5])


임베딩 벡터 길이: 768
임베딩 벡터 일부: [-0.017458646, 0.023586035, 0.0074004014, -0.09534135, 0.011647189]


### KF-DeBERTa embedding 모델

In [42]:
from langchain.embeddings.base import Embeddings
from transformers import AutoTokenizer, AutoModel
import torch

EB_MODEL = "kakaobank/kf-deberta-base"  # KF-DeBERTa 모델 이름
# KF-DeBERTa 임베딩 래퍼 정의
class KFDeBERTaEmbeddings(Embeddings):
    def __init__(self, model_name=EB_MODEL):
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.model = AutoModel.from_pretrained(model_name)
        self.device = "cuda" if torch.cuda.is_available() else "cpu"
        self.model.to(self.device)

    def embed_documents(self, texts):
        return [self.embed_query(text) for text in texts]

    def embed_query(self, text):
        inputs = self.tokenizer(text, return_tensors="pt", truncation=True, max_length=512, padding=True)
        inputs = {k: v.to(self.device) for k, v in inputs.items()}
        with torch.no_grad():
            outputs = self.model(**inputs)
            attention_mask = inputs["attention_mask"]
            token_embeddings = outputs.last_hidden_state
            input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
            embedding = torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)
        return embedding[0].cpu().numpy()

# 생성
embedding = KFDeBERTaEmbeddings()

sample_text = "금융권 문서 임베딩 예제"
embedding_vector = embedding.embed_query(sample_text)

print("임베딩 벡터 길이:", len(embedding_vector))
print("임베딩 벡터 일부:", embedding_vector[:5])

임베딩 벡터 길이: 768
임베딩 벡터 일부: [ 0.84986967 -0.74095684  0.1062606  -0.3817094   0.35903543]


### bge-m3 Embedding model

In [54]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter

EB_MODEL = "BAAI/bge-m3"
# 임베딩 모델 생성
embedding = HuggingFaceEmbeddings(
    model_name=EB_MODEL,
    model_kwargs={"device": "cpu"},  # GPU 사용 시
    encode_kwargs={"normalize_embeddings": True}
)

# sample_text = "금융권 문서 임베딩 예제"
# embedding_vector = embedding.embed_query(sample_text)

# print("임베딩 벡터 길이:", len(embedding_vector))
# print("임베딩 벡터 일부:", embedding_vector[:5])

## 청킹 방법 설정하기

### splitter 설정

In [81]:
# pip install langchain_experimental
if splitter_name == "recursive" :
    # recursive_character_text_splitter로 구분자 재귀 청킹 기법
    from langchain_text_splitters import RecursiveCharacterTextSplitter

    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000,
        chunk_overlap=100
    )
elif splitter_name == "semantic":
    # pip install langchain_experimental
    from langchain_experimental.text_splitter import SemanticChunker

    text_splitter = SemanticChunker(embedding)

print(text_splitter, "\nembedding: ", EB_MODEL)

embedding:  stsb-xlm-r-multilingual


### Debug 함수

In [82]:
def debug_chunkinfo_aftersplit(all_splits):
    for i, split in enumerate(all_splits):
        print(f"Chunk {i+1}:")
        print(split.page_content)
        print("-" * 40)

### 추출 하기

- pdf에서 추출

In [83]:
import os
import glob
from langchain_community.document_loaders import PyPDFLoader

# PDF 파일을 읽어서 텍스트 데이터 추출 및 청킹
def extract_documents_from_pdf(pdf_path):
    # PDF 파일을 읽어서 텍스트 데이터 추출
    loader = PyPDFLoader(pdf_path)
    data_nyc = loader.load()

    # 추출된 텍스트 데이터를 청킹
    splits = text_splitter.split_documents(data_nyc)
    # debug_chunkinfo_aftersplit(splits) 

    # recursive_character_text_splitter의 경우, 청크가 겹치는 부분이 없으면 연결하지 않음
    # 따라서, 청크가 겹치는 부분이 없을 때는 직접 연결하여 오버랩을 만듦
    # 만약 청크가 겹치는 부분이 있다면, 그 부분은 자동으로 연결됨
    # 여기서는 청크가 겹치지 않는 경우에만 오버랩을 추가함

    # 만약 첫 번째 청크의 끝과 두 번째 청크의 시작이 겹치지 않는다면,
    if splitter_name == "recursive" and (splits[0].page_content[-100:] == splits[1].page_content[:100]):
        print(splits[0].page_content)
        print("----")
        print(splits[1].page_content)
        for i in range(len(splits) - 1):
            splits[i].page_content += "\n" + splits[i + 1].page_content[:50]

    return splits


## 텍스트를 벡터로 변환하기

- 토큰 제한하며 실행 및 저장

In [84]:
# pip install pypdf
import tiktoken
import pypdf

try:
    encoding = tiktoken.encoding_for_model(EB_MODEL)
except KeyError:
    encoding = tiktoken.get_encoding("cl100k_base") # 토크나이저를 직접 지정


TOKEN_LIMIT_PER_BATCH = 39000  # 적절한 토큰 제한

def batch_save(vectorstore, splits):
    current_batch = []
    current_tokens = 0

    for doc in splits: 
        tokens = len(encoding.encode(doc.page_content))
        
        if current_tokens + tokens > TOKEN_LIMIT_PER_BATCH:
            try:
                vectorstore.add_documents(current_batch)
                print(f"✅ {len(current_batch)}개 문서 배치 저장 완료")
            except Exception as e:
                print(f"❌ 배치 저장 중 오류 발생: {e}")
            current_batch = [doc]
            current_tokens = tokens
        else:
            current_batch.append(doc)
            current_tokens += tokens

    # 마지막 배치 처리
    if current_batch:
        vectorstore.add_documents(current_batch)
        print(f"✅ 마지막 배치 {len(current_batch)}개 문서 저장 완료")
    
    return vectorstore

### Vector DB 생성 혹은 데이터 불러오기 (Chroma, FAISS)

In [85]:
# pip install faiss-cpu
import faiss
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.vectorstores import FAISS
from langchain.docstore.in_memory import InMemoryDocstore
from langchain_chroma import Chroma
import shutil
import time

persist_directory = './' + VECTOR_DB + '_' + splitter_name + '_' + EB_MODEL.replace('/', '_')

if not os.path.exists(persist_directory):
    print(f"Creating new {VECTOR_DB} at {persist_directory}...")    

    # 만약 persist_directory가 존재하지 않는다면, 새로 생성. data에 있는 PDF 파일을 읽어서 청킹 후 저장
    current_dir = os.getcwd() # 현재 폴더 경로
    folder_path = os.path.join(current_dir, "data") # data 폴더 경로 설정
    pdf_files = glob.glob(os.path.join(folder_path, "*.pdf")) # PDF 파일 목록 가져오기

    all_splits = []
    print(f"현재 청킹 방법 : {splitter_name}")
    print(f"현재 임베딩 모델 : {EB_MODEL}")
    print(f"PDF 파일 개수: {len(pdf_files)}")
    for pdf_file in pdf_files:
        pdf_path = pdf_file
        # temp_docs, merged_path = extract_documents_from_pdf(pdf_file)
        print(f"Processing {pdf_path}...")
        all_splits.extend(extract_documents_from_pdf(pdf_file))
        time.sleep(2)  # PDF 파일 처리 간에 잠시 대기

    print(f"전체 청크 개수: {len(all_splits)}")

    if VECTOR_DB == "chroma_store":
        vectorstore = Chroma(
            embedding_function=embedding,
            persist_directory=persist_directory
        )
        vectorstore = batch_save(vectorstore, all_splits)
    elif VECTOR_DB == "faiss_index":
    
        embedding_dim = len(embedding.embed_query("test"))
        index = faiss.IndexFlatIP(embedding_dim)

        vectorstore = FAISS(
            embedding_function=embedding.embed_query,
            index=index,
            docstore=InMemoryDocstore(),
            index_to_docstore_id={}
        )
        vectorstore = batch_save(vectorstore, all_splits)
        vectorstore.save_local(persist_directory)
else :
    # 이미 존재하는 Chroma store를 불러오기
    print(f"Loading existing {VECTOR_DB}...")
    if VECTOR_DB == "faiss_index":
        vectorstore = FAISS.load_local(persist_directory, embedding, allow_dangerous_deserialization=True)
    elif VECTOR_DB == "chroma_store":
        vectorstore = Chroma(
            embedding_function=embedding,
            persist_directory=persist_directory
        )

    # 만약 새로 추가할 PDF 파일이 있다면 데이터 추출 및 저장후, extra_data 폴더에서 data 폴더로 이동
    src_folder = 'extra_data'
    dst_folder = 'data'

    current_dir = os.getcwd() # 현재 폴더 경로
    folder_path = os.path.join(current_dir, src_folder) # extra_data 폴더 경로 설정
    pdf_files = glob.glob(os.path.join(folder_path, "*.pdf")) # PDF 파일 목록 가져오기

    extra_splits = []
    if not pdf_files:
        print("추가할 PDF 파일이 없습니다.")
    else:
        print(f"추가할 PDF 파일 개수: {len(pdf_files)}")
        for pdf_file in pdf_files:
            print(f"Processing {pdf_file}...")
            extra_splits.extend(extract_documents_from_pdf(pdf_file))
            time.sleep(1)

        vectorstore = batch_save(vectorstore, extra_splits)
        if VECTOR_DB == "faiss_index":
            vectorstore.save_local(persist_directory)
        # extra_data 폴더의 모든 파일 중 .pdf만 이동
        for filename in os.listdir(src_folder):
            if filename.lower().endswith('.pdf'):
                src_path = os.path.join(src_folder, filename)
                dst_path = os.path.join(dst_folder, filename)
                if os.path.isfile(src_path):
                    shutil.move(src_path, dst_path)

# semantic text-embedding-3-large chroma 12m 23.7s 청크 개수 342
# semantic stsb-xlm-r-multilingual chroma 2m 38.5s 청크 개수 342
# semantic intfloat/multilingual-e5-large-instruct chroma 6m 29.6s 청크 개수 342
# semantic kakaobank/kf-deberta-base chroma 9m 53.6s 청크 개수 342
# semantic BAAI/bge-m3 chroma 31m 48.9s 청크 개수 342



# semantic text-embedding-3-large faiss_index 12m 21.9s 청크 개수 342
# semantic stsb-xlm-r-multilingual faiss_index 3m 35.2s 청크 개수 342
# semantic intfloat/multilingual-e5-large-instruct faiss_index 7m 54.1s 청크 개수 342
# semantic kakaobank/kf-deberta-base faiss_index 9m 53.4s 청크 개수 342
# semantic BAAI/bge-m3 faiss_index 25m 3.8s 청크 개수 342

Loading existing faiss_index...
추가할 PDF 파일이 없습니다.


## LLM 설정 후 질문 및 답변

### gpt-4o-mini 설정

In [86]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_openai import ChatOpenAI
# from langchain_openai import ChatOpenAI # 라고 했을 때는 module 'openai' has no attribute 'DefaultHttpxClient' 오류가
from openai import OpenAI

if LLM_MODEL == "gpt-4o-mini":
    chat = ChatOpenAI(model=LLM_MODEL)
# elif LLM_MODEL == "gpt-oss-120b":
#     HF_API_KEY = os.getenv("HF_API_KEY")
#     chat = ChatOpenAI(
#         base_url="https://router.huggingface.co/v1",
#         api_key=HF_API_KEY,
#         model = LLM_MODEL
#     )

question_answering_promt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            # "You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Please write your answer in a markdown table format with the main points. Be sure to include all your source and page numbers like (3 ~ 10) in your answer. If you have over one source, you should include all of them. Answer in Korean. \n#Example Format: \n(brief summary of the answer) \n (table) \n  (detailed answer to the question) \n**출처** \n- (file source) (page source and page number) (Please write the quoted text within 20 characters and follow it with ... )\n #Context: {context}",
            "You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. you should user performed_context. You should consider all of special situation and general situation. If the 'perforemd_context' is null, you just say 'it is empyt'. Please write your answer in a markdown table format with the main points. Be sure to include all your source and page numbers like (3 ~ 10) in your answer. If you have over one source, you should include all of them. Answer in Korean. Also please write the keywords on user question that you think. \n#Example Format: \n(brief summary of the answer) \n (table) \n  (detailed answer to the question) \n**출처** \n- (file source) (page source and page number) (Please write the quoted text within 20 characters and follow it with ... )\n\n 키워드 : (keywords)\n #Context: {context}",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

document_chain = create_stuff_documents_chain(chat, question_answering_promt)

- meta data에 있는 정보는 llm이 알 수 없으므로 출처 filename을 포함시켜서 새로운 Document 타입을 생성

In [87]:
import os
from langchain.schema import Document

# Document type인 docs가 context로 넘겨졌을 때 llm은 content에 있는 정보 기반으로 답을 한다.
# 따라서 meta data에 있는 정보는 llm이 알 수 없으므로 출처 filename을 포함시켜서 새로운 Document 타입을 생성한다.
def format_docs_with_source_as_documents(docs):
    new_docs = []
    for d in docs:
        filename = os.path.basename(d.metadata.get("source", ""))
        # 기존 page_content 뒤에 출처 붙이기
        new_content = f"{d.page_content}\n출처: {filename}"

        # 새 리스트 생성 (metadata 유지)
        new_docs.append(
            Document(page_content=new_content, metadata=d.metadata)
        )
    return new_docs



### 질문 입력 및 답변 생성

In [88]:

from openai import OpenAI
load_dotenv()
HF_API_KEY = os.getenv("HF_API_KEY")
client = OpenAI(
    base_url="https://router.huggingface.co/v1",
    api_key=HF_API_KEY
)
question = "우리 부부의 연소득은 총 합 6800만원이야. 한 명은 군에 종사하고 하나는 직장인이라는 것을 참고해줘. 받을 수 있는 대출은 뭐가 있을까?"

# "우리 부부의 연소득은 총 합 6800만원이야. 한 명은 군에 종사하고 하나는 직장인이라는 것을 참고해줘. 받을 수 있는 대출은 뭐가 있을까?"
# '"연소득", "6800만원", "군인", "직장인", "대출", "부부"'
# "우리 부부의 연소득은 총 합 6800만원이야. 한 명은 군에 종사하고 하나는 직장인이라는 것을 참고해줘. 받을 수 있는 대출은 뭐가 있을까?"
# '"연소득", "6800만원", "군인", "직장인", "대출", "부부"'
# "우리 부부의 연소득은 총 합 6800만원이야. 한 명은 군에 종사하고 하나는 직장인이라는 것을 참고해줘. 받을 수 있는 대출은 뭐가 있을까?"
# "우리 부부는 한 명은 군인이고 한 명은 직장인이야. 전세 때문에 대출을 받으려고해. 둘 합산 연소득은 6000만원이야 받을 수 있는 대출은 뭐가 있지? 모두 알려줘"
# "우리 부부의 연소득은 총 합 6800만원이야. 받을 수 있는 대출은 뭐가 있을까?"

# print(formatted_context)
summary_completion = client.chat.completions.create(
    model="openai/gpt-oss-120b",
    messages=[{
        "role": "user", 
        "content": f"You are an assistant for question-answering tasks. 벡터 db에 넣기 위해 주어진 질문의 키워드를 파이썬 리스트 형태로 답변해줘. 백틱도 안 넣어도돼. The question is : {question}"
        }],
)
# print("질문:", question)
# print("답변:")
# print("-" * 40)

question_for_vectordb = summary_completion.choices[0].message.content
print(question_for_vectordb)




["연소득", "6800만원", "군인", "직장인", "부부", "대출", "가능한 대출"]


In [89]:
retriever = vectorstore.as_retriever(search_kwargs={'k': 10})
# k를 3개에서 10개로 변경

docs = retriever.invoke(question_for_vectordb)
# print(type(docs))
# print(docs)
formatted_context = format_docs_with_source_as_documents(docs)

for d in formatted_context:
    print(d.metadata["source"])
    # print(d)
    print("-" * 40)
# for d in docs:
#     print(d.metadata)
#     print(d.page_content)
#     print("-" * 40)


c:\ITStudy\Project\TechSeminar_Public\by_book\data\적격대출_업무처리기준.pdf
----------------------------------------
c:\ITStudy\Project\TechSeminar_Public\by_book\data\bogeum_guidline.pdf
----------------------------------------
c:\ITStudy\Project\TechSeminar_Public\by_book\data\디딤돌대출_업무처리기준.pdf
----------------------------------------
c:\ITStudy\Project\TechSeminar_Public\by_book\data\적격대출_업무처리기준.pdf
----------------------------------------
c:\ITStudy\Project\TechSeminar_Public\by_book\data\적격대출_업무처리기준.pdf
----------------------------------------
c:\ITStudy\Project\TechSeminar_Public\by_book\data\디딤돌대출_업무처리기준.pdf
----------------------------------------
c:\ITStudy\Project\TechSeminar_Public\by_book\data\디딤돌대출_업무처리기준.pdf
----------------------------------------
c:\ITStudy\Project\TechSeminar_Public\by_book\data\적격대출_업무처리기준.pdf
----------------------------------------
c:\ITStudy\Project\TechSeminar_Public\by_book\data\디딤돌대출_업무처리기준.pdf
----------------------------------------
c:\ITStudy\Project\T

### openai/gpt-oss-120b 모델 사용

In [90]:

from openai import OpenAI
load_dotenv()
HF_API_KEY = os.getenv("HF_API_KEY")
client = OpenAI(
    base_url="https://router.huggingface.co/v1",
    api_key=HF_API_KEY
)
# print(formatted_context)
summary_completion = client.chat.completions.create(
    model="openai/gpt-oss-120b",
    messages=[{
        "role": "user", 
        "content": f"You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. you should user performed_context. You should consider all of special situation and general situation. If the 'perforemd_context' is null, you just say 'it is empyt'. Please write your answer in a markdown table format with the main points. Be sure to include all your source and page numbers like (3 ~ 10) in your answer. If you have over one source, you should include all of them. Answer in Korean. Also please write the keywords on user question that you think. \n#Example Format: \n(brief summary of the answer) \n (table) \n  (detailed answer to the question) \n**출처** \n- (file source) (page source and page number) (Please write the quoted text within 20 characters and follow it with ... )\n\n 키워드 : (keywords)\n #Context: {formatted_context}. The question is : {question}"
        }],
)

# print("답변:")
# print("-" * 40)

# print("질문:", question)
print("|분류|종류|")
print("|---|---|")
print("|청킹 방법|", splitter_name, "|")
print("|임베딩 모델|", EB_MODEL, "|")
print("|벡터 DB|", VECTOR_DB, "|")
print("|LLM 모델|", "openai/gpt-oss-120b", "|\n")

print("답변 : ")

print(summary_completion.choices[0].message.content)
print("\n\n")
print("-" * 40)



|분류|종류|
|---|---|
|청킹 방법| semantic |
|임베딩 모델| stsb-xlm-r-multilingual |
|벡터 DB| faiss_index |
|LLM 모델| openai/gpt-oss-120b |

답변 : 
**요약**  
- **군인·퇴직금 기반 대출** – 군인 전용·우대 대출(퇴직금대출·추가 대출) 가능 → 최고 8 천만원(대령 이상) 한도, 10 년 이내 상환.  
- **내집마련·디딤돌대출** – 연소득 6 800만원(부부 합산) 기준, LTV ≤ 70 % (생애 최초 구매자는 80 %까지) 적용, DTI ≤ 40 %~50 % 수준이면 허용.  
- **일반 적격주택담보대출** – 동일 LTV·DTI 기준 적용, 신용점수·연령·무주택 검증 충족 시 신청 가능.  

| 대출 종류 | 주요 조건 / 한도 | 연소득·가족 구성에 대한 적용 | 비고 | 출처 |
|-----------|------------------|------------------------------|------|------|
| **군인·퇴직금 대출** (우리은행) | • 퇴직수당·퇴직금 예상액 범위 내 대출 <br>• 복무 25년 이상·원사·준위·중령 이상 → 최고 8 천만원 (대령·그 이상은 복무 기간 제한 없음) <br>• 2 천만원 이내는 퇴직급여 예상액 범위 내 | 군인인 배우자(또는 본인)의 퇴직급여·퇴직수당 예상액을 기준으로 한도 산정. 연소득 6 800만원은 대출 심사시 DTI·DTI 계산에 활용됨. | 대출기간: 최장 10년(만기일시상환·분할상환 모두 가능) <br>※ 추가 대출은 기존 퇴직금 대출 수령 고객에 한해, ‘퇴직급여 예상액’ 범위 내. | 우리 군인우대 대출.pdf (page 2) |
| **디딤돌대출(내집마련)** | • LTV ≤ 70 % (생애 최초 주택구입자는 최대 80 %·수도권·규제지역은 70 %) <br>• DTI 계산 시 부부 합산소득 사용 <br>• DTI ≤ 40 %~50 % 정도(정확한 기준은 금융기관 내부

### lanchain 사용

In [91]:
from langchain.memory import ChatMessageHistory

chat_history = ChatMessageHistory()

chat_history.add_user_message(question)

answer = document_chain.invoke(
    {
        "messages": chat_history.messages,
        "context": formatted_context,
    }
)

chat_history.add_ai_message(answer)

print("|분류|종류|")
print("|---|---|")
print("|청킹 방법|", splitter_name, "|")
print("|임베딩 모델|", EB_MODEL, "|")
print("|벡터 DB|", VECTOR_DB, "|")
print("|LLM 모델|", LLM_MODEL, "|\n")

print("답변 : ")

print(answer)

print("\n\n")
print("-" * 40)

|분류|종류|
|---|---|
|청킹 방법| semantic |
|임베딩 모델| stsb-xlm-r-multilingual |
|벡터 DB| faiss_index |
|LLM 모델| gpt-4o-mini |

답변 : 
| 대출 종류    | 자격 요건 및 조건                                            |
|---------------|-------------------------------------------------------|
| 디딤돌 대출   | - 연소득 6800만원 이내 가능 (부부 합산 소득 기준) <br> - 본건 대출 금액과 선순위 채권의 합이 주택 평가액의 70% 이내 (일반 LTV) <br> - 군인은 추가 대출 가능                    |
| 군인 퇴직금 대출 | - 군인으로서 복무하는 경우, 퇴직금 예상액 범위 내에서 대출 가능 <br> - 조건에 따라 대출한도 별도 적용 가능                 |

**자세한 설명**  
총 연소득이 6800만원인 부부의 경우 '디딤돌 대출'을 이용할 수 있습니다. 이 대출은 연소득이 6800만원 이내일 때 신청할 수 있으며, 대출 금액은 주택 평가액의 70% 이내여야 합니다. 군에 종사하는 분은 퇴직금 대출 가능성 또한 존재하며, 이는 퇴직금 예상액 범위 내에서 대출을 받을 수 있습니다.

**출처**
- 디딤돌대출_업무처리기준.pdf (1 페이지)
- 우리 군인우대 대출.pdf (7 페이지)

키워드 : 대출, 연소득, 디딤돌 대출, 군인 퇴직금 대출



----------------------------------------


### context 사용 없이 질문을 받았을 경우

In [ ]:

from openai import OpenAI
from langchain_openai import ChatOpenAI

load_dotenv()
HF_API_KEY = os.getenv("HF_API_KEY")
client = OpenAI(
    base_url="https://router.huggingface.co/v1",
    api_key=HF_API_KEY
)
# print(formatted_context)
summary_completion = client.chat.completions.create(
    model="openai/gpt-oss-120b",
    messages=[{
        "role": "user", 
        "content": f"You are an assistant for question-answering tasks. Please keep the answer under 500 characters. Please write your answer in a markdown table format with the main points.  Answer in Korean. \n#Example Format: \n(brief summary of the answer) \n (table) \n  (detailed answer to the question) \n**출처** \n- (file source) (page source and page number and correct full url) (Please write the quoted text within 20 characters and follow it with ... )\n The question is : {question}"
        }],
)
print("\n청킹 방법", "context 사용 없음")
print("임베딩 모델", "context 사용 없음")
print("LLM 모델", "openai/gpt-oss-120b")
print("벡터 DB", "사용 없음")

print("답변 : ")
print("-" * 40)
print(summary_completion.choices[0].message.content)

질문: 우리 부부는 한 명은 군에 종사하고 있고, 한 명은 직장인이야. 전세 때문에 대출을 받으려고해. 둘 합산 연소득은 6800만원이야 받을 수 있는 대출은 뭐가 있지? 모두 알려줘
답변:
----------------------------------------
**요약**: 연소득 6,800만원이면 전세자금대출·주택담보대출·신용대출 등을 활용해 2 ~ 3억 원까지 대출이 가능해요.  

|대출 종류|주요 조건|예상 한도(최대)|
|---|---|---|
|전세자금 대출|소득·신용·주택가격·전세보증금 기준, LTV 80%·DTI 45% 이하|전세보증금 80%·≈2.5억|
|주택담보대출|주택소유·LTV 70%·DTI 50% 이하|주택가치 70%·≈3억|
|신용(일반)대출|소득·신용점수·부채비율 ≤ 60%|연소득 30%·≈2억|

전세자금 대출은 은행·주택금융공사가 공동으로 제공하고, 주택담보대출은 보유 주택이 있을 경우 LTV·DTI에 따라 한도가 결정됩니다. 신용대출은 부채·신용점수에 따라 연소득 30% 정도까지 제한됩니다.

**출처**  
- 금융감독원, “주택대출·DTI·LTV 기준” (2024) https://fss.or.kr/... (“LTV 80%·DTI 45%...” )  
- KB국민은행, “전세자금 대출상품 안내” https://obank.kbstar.com/... (“전세보증금 80%…” )

청킹 방법 context 사용 없음
임베딩 모델 context 사용 없음
LLM 모델 openai/gpt-oss-120b


## 질의 확장 구현
- 챗본으로 만들 경우 첫번째 질문을 이어 받아서 두번째 질문도 답변을 해야하니 질의 확장에 대한 기능이다. (필수 x)

In [37]:
from langchain_core.output_parsers import StrOutputParser

query_for_other = "3천만원 일때는?"

query_augmentation_prompt = ChatPromptTemplate.from_messages(
    [
        MessagesPlaceholder(variable_name="messages"),
        (
            "system",
            "기존의 대화 내용을 활용하여 사용자가 질문한 의도를 파악해서 한 문장의 명료한 질문으로 변환하라. 대명사나 이, 저, 그와 같은 표현을 명확한 명사로 표현하라. : \n\n{query}",
        ),
    ]
)
query_augmentation_chain = query_augmentation_prompt | chat | StrOutputParser()

augmented_query = query_augmentation_chain.invoke(
    {
        "messages": chat_history.messages,
        "query": query_for_other,
    }
)

print(augmented_query)

부부의 연소득이 3000만원일 때 받을 수 있는 대출은 무엇인가요?


In [ ]:
docs_other = retriever.invoke(augmented_query)
# print(type(docs))
# print(docs)
formatted_context_other = format_docs_with_source_as_documents(docs_other)

chat_history.add_user_message(query_for_other)

answer = document_chain.invoke(
    {
        "messages": chat_history.messages,
        "context": formatted_context_other,
    }
)

chat_history.add_ai_message(answer)

print("질문:", query_for_other)
print("답변:")
print("-" * 40)
print(answer)

print("\n청킹 방법", splitter_name)
print("임베딩 모델", EB_MODEL)
print("LLM 모델", LLM_MODEL)

질문: 3천만원 일때는?
답변:
----------------------------------------
연소득이 3000만원일 때의 대출 가능성은 다음과 같이 요약할 수 있습니다.

| 주요 내용               | 세부 사항                                     |
|----------------------|--------------------------------------------|
| 연소득               | 3000만원                                    |
| 대출한도 — 일반      | 2억원 이내                                   |
| 대출한도 — 생애최초  | 2.4억원                                     |
| 대출한도 — 신혼·다자녀 | 3.2억원                                     |
| 소득 추정 기준       | 5천만원 이하 대출 취급 가능                 |

부부의 연소득이 3000만원인 경우, 일반적인 대출한도는 최대 2억원입니다. 또한 생애 최초 주택 구입자나 신혼 및 다자녀 가구의 경우 각각 2.4억원과 3.2억원까지 대출 가능성이 있습니다. 소득이 5천만원 이하이므로 대출 신청이 가능하다는 점 또한 주요합니다.

**출처**
- 디딤돌대출_업무처리기준.pdf (22페이지) "소득추정 금액이 60백만원 이하 ..." 

청킹 방법 recursive
임베딩 모델 text-embedding-3-large
LLM 모델 gpt-4o-mini
